<a href="https://colab.research.google.com/github/deryacoban/final-project/blob/derya/bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# YOLOv6'yı GitHub'dan klonlayın
!git clone https://github.com/meituan/YOLOv6.git
%cd YOLOv6

# Gerekli kütüphaneleri yükleyin
!pip install -r requirements.txt


Cloning into 'YOLOv6'...
remote: Enumerating objects: 3838, done.
remote: Total 3838 (delta 0), reused 0 (delta 0), pack-reused 3838 (from 1)
Receiving objects: 100% (3838/3838), 43.99 MiB | 32.26 MiB/s, done.
Resolving deltas: 100% (2297/2297), done.
/content/YOLOv6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.2 MB/s eta 0:00:00


In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Datasetinizi Colab çalışma alanına kopyalayın



Mounted at /content/drive


In [4]:
!pwd  # Geçerli dizini kontrol eder
!ls   # Geçerli dizindeki dosyaları listeler


/content/YOLOv6
assets	 data	 docs	     inference.ipynb  README_cn.md  requirements.txt  tutorial.ipynb
configs  deploy  hubconf.py  LICENSE	      README.md     tools	      yolov6


In [5]:
!ls configs


base	    yolov6l6_finetune.py  yolov6m6_finetune.py	yolov6n6.py	      yolov6s_finetune.py
experiment  yolov6l6.py		  yolov6m6.py		yolov6n_finetune.py   yolov6s.py
mbla	    yolov6l_finetune.py   yolov6m_finetune.py	yolov6n.py
qarepvgg    yolov6_lite		  yolov6m.py		yolov6s6_finetune.py
repopt	    yolov6l.py		  yolov6n6_finetune.py	yolov6s6.py


In [ ]:
!cat configs/yolov6s.py


# YOLOv6s model
model = dict(
    type='YOLOv6s',
    pretrained=None,
    depth_multiple=0.33,
    width_multiple=0.50,
    backbone=dict(
        type='EfficientRep',
        num_repeats=[1, 6, 12, 18, 6],
        out_channels=[64, 128, 256, 512, 1024],
        fuse_P2=True,
        cspsppf=True,
        ),
    neck=dict(
        type='RepBiFPANNeck',
        num_repeats=[12, 12, 12, 12],
        out_channels=[256, 128, 128, 256, 256, 512],
        ),
    head=dict(
        type='EffiDeHead',
        in_channels=[128, 256, 512],
        num_layers=3,
        begin_indices=24,
        anchors=3,
        anchors_init=[[10,13, 19,19, 33,23],
                      [30,61, 59,59, 59,119],
                      [116,90, 185,185, 373,326]],
        out_indices=[17, 20, 23],
        strides=[8, 16, 32],
        atss_warmup_epoch=0,
        iou_type='giou',
        use_dfl=False, # set to True if you want to further train with distillation
        reg_max=0, # set to 16 if you want to further

In [19]:
!python tools/train.py \
  --conf /content/YOLOv6/configs/yolov6s.py \
  --data /content/YOLOv6/data/dataset.yaml \
  --device cpu \
  --epochs 50 \
  --batch-size 8




2025-01-15 21:29:31.098341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-15 21:29:31.117983: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-15 21:29:31.123891: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 21:29:31.138061: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-15 21:29:32.235790: W tensorflow/compiler/tf2

In [18]:
import glob, os

# Kendi dataset yolunuza uyarlayın:
label_dir = '/content/YOLOv6/dataset/labels/train'
nc = 2  # dataset.yaml içindeki sınıf sayısı

files = glob.glob(os.path.join(label_dir, '*.txt'))
for f in files:
    with open(f, 'r') as fp:
        lines = fp.readlines()
        for line in lines:
            vals = line.strip().split()
            if len(vals) == 5:
                class_id = int(vals[0])
                x_center = float(vals[1])
                y_center = float(vals[2])
                width    = float(vals[3])
                height   = float(vals[4])
                # Sınıf aralığı kontrolü
                if class_id < 0 or class_id >= nc:
                    print(f"Hatalı class_id={class_id} --> {f}")
                # 0-1 aralığı kontrolü
                if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                    print(f"Hatalı box değerleri: {line.strip()} --> {f}")


In [22]:
!python tools/train.py --conf configs/yolov6s.py --data data/dataset.yaml --epochs 50 --batch-size 2 --device cpu



2025-01-15 21:35:37.223784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-15 21:35:37.243586: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-15 21:35:37.249504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 21:35:37.263764: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-15 21:35:38.310543: W tensorflow/compiler/tf2

In [27]:
import json

with open('/content/drive/MyDrive/dataset/annotations/instances_val.json') as f:
    data = json.load(f)

for annotation in data['annotations']:
    bbox = annotation['bbox']
    if not (0 <= bbox[0] <= 1 and 0 <= bbox[1] <= 1 and bbox[2] > 0 and bbox[3] > 0):
        print(f"Hatalı bounding box bulundu: {bbox}")


Hatalı bounding box bulundu: [248.0000056028366, 137.9997808933258, 148.99961042404175, 484.000216960907]
Hatalı bounding box bulundu: [543.9997348487377, 120.00007963180542, 178.0001454949379, 493.0000972747803]
Hatalı bounding box bulundu: [429.0006129294634, 363.00016272068024, 87.99958857893944, 230.0002944469452]
Hatalı bounding box bulundu: [278.0005202293396, 198.99980521202087, 133.99984288215637, 376.9999556541443]
Hatalı bounding box bulundu: [576.0002160072327, 166.0003364086151, 159.0004448890686, 392.999707698822]
Hatalı bounding box bulundu: [433.00053115189075, 310.0003310441971, 87.99958857893944, 257.9998605251312]
Hatalı bounding box bulundu: [431.00007113814354, 57.999714612960815, 113.0001831650734, 574.0001282691956]
Hatalı bounding box bulundu: [560.000023663044, 91.00021243095398, 134.99983727931976, 543.9999203681946]
Hatalı bounding box bulundu: [263.9997489899397, 316.9998861551285, 105.00030961632729, 281.0001571178436]
Hatalı bounding box bulundu: [415.00032

In [35]:
import cv2
import os
from PIL import Image

label_folder = '/content/drive/MyDrive/dataset/labels/'
image_folder = '/content/drive/MyDrive/dataset/images/'

label_files = [f for f in os.listdir(label_folder) if f.endswith('.txt')]

for label_file in label_files:
    label_path = os.path.join(label_folder, label_file)
    image_path = os.path.join(image_folder, label_file.replace('.txt', '.jpg'))

    if not os.path.exists(image_path) or not os.path.isfile(image_path):
        print(f"Resim bulunamadı: {image_path}")
        continue

    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        cls = int(parts[0])
        x_center, y_center, width, height = map(float, parts[1:])

        # Bounding box koordinatlarını denormalize et
        x_center = int(x_center * image_width)
        y_center = int(y_center * image_height)
        width = int(width * image_width)
        height = int(height * image_height)

        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # Bounding box'u çiz
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, str(cls), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Bounding box'lı resmi göster
    cv2.imshow("Bounding Boxes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()






In [36]:
import json

with open('/content/drive/MyDrive/dataset/annotations/instances_val.json') as f:
    data = json.load(f)

for annotation in data['annotations']:
    bbox = annotation['bbox']
    if not (0 <= bbox[0] <= 1 and 0 <= bbox[1] <= 1 and bbox[2] > 0 and bbox[3] > 0):
        print(f"Hatalı bounding box bulundu: {bbox}")




Hatalı bounding box bulundu: [248.0000056028366, 137.9997808933258, 148.99961042404175, 484.000216960907]
Hatalı bounding box bulundu: [543.9997348487377, 120.00007963180542, 178.0001454949379, 493.0000972747803]
Hatalı bounding box bulundu: [429.0006129294634, 363.00016272068024, 87.99958857893944, 230.0002944469452]
Hatalı bounding box bulundu: [278.0005202293396, 198.99980521202087, 133.99984288215637, 376.9999556541443]
Hatalı bounding box bulundu: [576.0002160072327, 166.0003364086151, 159.0004448890686, 392.999707698822]
Hatalı bounding box bulundu: [433.00053115189075, 310.0003310441971, 87.99958857893944, 257.9998605251312]
Hatalı bounding box bulundu: [431.00007113814354, 57.999714612960815, 113.0001831650734, 574.0001282691956]
Hatalı bounding box bulundu: [560.000023663044, 91.00021243095398, 134.99983727931976, 543.9999203681946]
Hatalı bounding box bulundu: [263.9997489899397, 316.9998861551285, 105.00030961632729, 281.0001571178436]
Hatalı bounding box bulundu: [415.00032